In [ ]:
"""
Objectivo::

    -> gerar contribuições ao longo de todo o incêndio
    a partir de um raster com o ARRIVAL_TIME;

As contribuições vão ser criadas iterativamente.
O número de iterações será igual ao número de 
intervalos tempo definidos a partir de três
inputs fornecidos pelo utilizador:
* inicio;
* fim;
* duração período.

O número de iterações pode ser calculado do seguinte modo:
(fim - início) / duração do período.

Como o raster ARRIVAL_TIME indica somente o momento em que o 
fogo chega a uma determinada célula, o utilizador 
tem de especificar um valor que indique quanto tempo
cada célula demora a arder.
Isto permitirá saber, em cada período, quais as áreas
que se encontram a arder.
"""

ARRIVAL_RST = '/home/jasp/fireloc/cmb_arrivalt.tif'
LMT_SHP = '/home/jasp/fireloc/lmt_coimbra.shp'
CELL_BURN_TIME = 8*60 # 8 Hours in minutes
FIRE_BEGIN = '2015-07-21 14:15:00'
FIRE_END   = '2015-07-30 08:00:00'
TIME_PERIOD = 30 # minutes

slug_dem = 'dem888_10'
dem_rst = '/home/jasp/fireloc/dem.tif'

import datetime as dt
import os; import json
from glass.g.rst.rcls import rcls_rst
from glass.pys.oss import mkdir
from glass.g.fmrst import rst_to_geodf
from glass.g.wenv.grs import run_grass
from glass.to.web import get_file_post
from glass.g.to import shpext_to_boundary
from glass.g.rst.zon import region_group
from glass.geo.prop.rst import frequencies

# Get Time Periods
fire_begin = dt.datetime.strptime(FIRE_BEGIN, '%Y-%m-%d %H:%M:%S')
fire_end   = dt.datetime.strptime(FIRE_END, '%Y-%m-%d %H:%M:%S')

fire_duration = fire_end - fire_begin
n_periods = int(round((fire_duration.total_seconds() / 60) / TIME_PERIOD, 0))

time_periods = [TIME_PERIOD * (i+1) for i in range(n_periods)]

In [ ]:
# Create temporary folder
tmp_fld = mkdir(os.path.join(os.path.dirname(ARRIVAL_RST), 'tmp'))

# Get DEM for computing visibility

if not dem_rst:
    # Get Extent Geometry
    lmt_geom = shpext_to_boundary(LMT_SHP, out_srs=3763).ExportToWkt()
    sdi_url = 'https://vgi.mat.uc.pt/api/sdi/down-dataset/'
    auth_dt = ('kurosaki', 'zangetsu')
    data = json.dumps({
        'geom' : lmt_geom, 'dataset' : slug_dem
    })

    status, dem_rst = get_file_post(sdi_url, os.path.join(
        tmp_fld, 'dem.tif'
    ), data, auth_dt)

In [ ]:
# Create GRASS GIS Session
gbase = run_grass(tmp_fld, location='loc', srs=dem_rst)

# Start GRASS GIS Session
import grass.script as grass
import grass.script.setup as gsetup
gsetup.init(gbase, tmp_fld, 'loc', 'PERMANENT')

from glass.dct.geo.torst import rst_to_grs, grs_to_rst
from glass.g.rst.surf import grs_viewshed
from glass.g.rst.alg import rstcalc
from glass.g.smp import rst_random_pnt
from glass.geo.df.toshp.cff import grs_to_shp

# Send DEM to GRASS GIS
grs_dem = rst_to_grs(dem_rst, 'grs_dem', as_cmd=True)

In [ ]:
def get_contrib(cbt, time_interval, rst_arrival):
    # ID Burned Areas
    rst_bareas = rcls_rst(rst_arrival, {
        (-1, time_interval) : 1,
        '*'                : 'NoData'
    } if time_interval <= cbt else {
        (time_interval- cbt, time_interval) : 1,
        '*'                : 'NoData'
    }, os.path.join(tmp_fld, 'burn_{}.tif'.format(str(time_interval))), maintain_ext=None)
    
    # Burned areas to Points
    df_bareas = rst_to_geodf(rst_bareas)
    df_bareas.reset_index(drop=True, inplace=True)
    
    # Get visible areas from Burned Areas
    # Burned Areas as observation points
    vis_rst = []
    for i, row in df_bareas.iterrows():
        if i > 2:
            break
        vis_rst.append(grs_viewshed(
            grs_dem, (row.geometry.x, row.geometry.y),
            'vis_{}'.format(str(i)),
            max_dist=5000 *2, obs_elv=200
        ))
    
    view_burn = rstcalc(
        "if(({}) > 0, 1, null())".format(" + ".join(vis_rst)), 
        'vis_{}'.format(str(time_interval)),
        api='grass'
    )
    # Controlo
    view_burn_exp = grs_to_rst(
        view_burn, os.path.join(tmp_fld, view_burn + '.tif'))
    
    # Group By Region
    grp_rst = region_group(view_burn, 'rgroup_{}'.format(str(time_interval)))
    grp_rst_exp = grs_to_rst(
        grp_rst, os.path.join(tmp_fld, grp_rst + '.tif'))
    
    # Count regions cells
    rst_freq = frequencies(grp_rst_exp)
    rst_val = list(rst_freq.keys())
    rst_val.sort()
    val_npnt = [int(
        (rst_freq[v] * 30) / 100
    ) if rst_freq[v] > 3 else 1 for v in rst_val]
    
    # Create Random points in Region
    # N pontos depende do tamanho da região
    rnd_pnt = rst_random_pnt(
        grp_rst, tuple(val_npnt), 'obs_{}'.format(str(time_interval)),
        xpnt_by_cat=True
    )
    
    end_shp = grs_to_shp(rnd_pnt, os.path.join(tmp_fld, rnd_pnt + '.shp'))
    
    # e ate da classe de LULC
    
    # Calcula tempo
    
    # Calcula direcção
    
    return 1

In [ ]:
time_a = dt.datetime.now().replace(microsecond=0)
print(time_a)
get_contrib(CELL_BURN_TIME, 30, ARRIVAL_RST)
time_b = dt.datetime.now().replace(microsecond=0)
print(time_b)
print(time_b - time_a)

In [ ]:
from osgeo import gdal
import numpy as np
from glass.g.fmrst import array_to_geodf
from glass.geo.df.toshp import df_to_shp
import datetime as dt

img = gdal.Open('/home/jasp/fireloc/tmp/rgroup_30.tif')

arr = img.ReadAsArray()
ndval = img.GetRasterBand(1).GetNoDataValue()

arr_one = arr.reshape(arr.shape[0] * arr.shape[1])
idx = np.arange(arr_one.shape[0]).astype(np.uint32)

rst_vals = np.unique(arr).astype(np.int32)
rst_vals = rst_vals[rst_vals != ndval]

res = np.zeros(arr_one.shape[0], dtype=np.uint8)

In [ ]:
time_a = dt.datetime.now().replace(microsecond=0)
print(time_a)

for val in rst_vals:
    i_idx = idx[arr_one == val]
    npnt = int(i_idx.shape[0] * 30 / 100) if i_idx.shape[0] > 3 else 1
    
    i_rnd = np.random.choice(i_idx, npnt)
    mask = np.isin(idx, i_rnd)
    
    np.place(res, mask, 1)

time_b = dt.datetime.now().replace(microsecond=0)
print(time_b)
print(time_b - time_a)

In [ ]:
res = res.reshape(arr.shape)

geodf = array_to_geodf(res, img.GetGeoTransform(), 3763, 0)

df_to_shp(geodf, '/home/jasp/fireloc/tmp/random_30.shp')